In [2]:
from open3d import *
import numpy as np
import copy
import matplotlib.pyplot as plt

In [3]:
def draw_registration_result(source, target, transformation):
    # Deep copy source
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)

    # Set color for each model
    # Orange
    source_temp.paint_uniform_color([1, 0.706, 0])
    # Blue
    target_temp.paint_uniform_color([0, 0.651, 0.914])
     
    # Offset Source model and Draw Geometries
    source_temp.transform(transformation)
    draw_geometries([source_temp, target_temp])

def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb initial pose.")
    
    ##########################################
    # Pass in path to point cloud here
    ##########################################
    source = read_point_cloud("seg_after.ply")
    target = read_point_cloud("seg_before.ply")

    trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0],
                            [1.0, 0.0, 0.0, 0.0],
                            [0.0, 1.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 1.0]])
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    print(":: Downsample with a voxel size %.3f." % voxel_size)
    source_down = voxel_down_sample(source, voxel_size)
    target_down = voxel_down_sample(target, voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)

    estimate_normals(source_down, KDTreeSearchParamHybrid(
            radius = radius_normal, max_nn = 30))
    estimate_normals(target_down, KDTreeSearchParamHybrid(
            radius = radius_normal, max_nn = 30))

    
    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    source_fpfh = compute_fpfh_feature(source_down,
            KDTreeSearchParamHybrid(radius = radius_feature, max_nn = 100))
    target_fpfh = compute_fpfh_feature(target_down,
            KDTreeSearchParamHybrid(radius = radius_feature, max_nn = 100))
    return source, target, source_down, target_down, source_fpfh, target_fpfh

def execute_global_registration(
        source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = registration_ransac_based_on_feature_matching(
            source_down, target_down, source_fpfh, target_fpfh, 0.075,
            TransformationEstimationPointToPoint(False), 4,
            [CorrespondenceCheckerBasedOnEdgeLength(0.9),
            CorrespondenceCheckerBasedOnDistance(0.075)],
            RANSACConvergenceCriteria(4000000, 500))
    return result

def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = registration_icp(source, target, distance_threshold,
            result_ransac.transformation,
            TransformationEstimationPointToPlane())
    return result

In [6]:
if __name__ == "__main__":
    voxel_size = 0.05 # means 5cm for the dataset
    source, target, source_down, target_down, source_fpfh, target_fpfh = \
            prepare_dataset(voxel_size)

    result_ransac = execute_global_registration(source_down, target_down,
            source_fpfh, target_fpfh, voxel_size)

    print("RanSac Results", result_ransac)

    draw_registration_result(source_down, target_down, result_ransac.transformation)

    result_icp = refine_registration(source, target,
            source_fpfh, target_fpfh, voxel_size)
    print("Refine Registration Results :", result_icp)

    print("\nFitness",result_icp.fitness)
    print("\nInlier Rmse",result_icp.inlier_rmse)
    print("\nCorrespondence Set", np.asarray(result_icp.correspondence_set))

    source_cord = np.asarray(source.points)
    print("Source cordinates", source_cord)
    
    draw_registration_result(source, target, result_icp.transformation)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
RanSac Results RegistrationResult with fitness = 1.000000, inlier_rmse = 0.023637, and correspondence_set size of 1302
Access transformation to get result.
:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.020.
Refine Registration Results : RegistrationResult with fitness = 0.916004, inlier_rmse = 0.009471, and correspondence_set size of 12432
Access transformation to get result.

Fitness 0.9160035366931919

Inlier Rmse 0.009470693862539872

Correspondence Set [[ 8484 28328]
 [ 8485 17375]
 [ 8486 17376]
 ...
 [ 5088 11123]
 [ 5089 20257]
 [ 5090 26678]]
Source cor